<a href="https://colab.research.google.com/github/SrijaG29/Pyspark_practise/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PySpark**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=889fb1841b82596524d52ffc1982ac64c4d8b108f48add839c3bf3c4de8d8503
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark import SparkConf,SparkContext

In [ ]:
conf = SparkConf().setAppName("Read File")
sc = SparkContext.getOrCreate(conf = conf)

Instead of dowloading the text file using path of the file we can directly create the text file here.

For writing the file.

In [ ]:
with open('sampletext.txt', 'w') as f:
  f.write('Hi How are you\n')
  f.write('Hope you are doing good\n')
  f.write('great')

For reading the file.

In [ ]:
with open('sampletext.txt','r') as f:
  x = f.read()
  print(x)

Hi How are you
Hope you are doing good
great


Fo direct download from the files in colab we need to give file path.

In [ ]:
text = sc.textFile('/content/sampletext.txt')
text.collect()

['Hi How are you', 'Hope you are doing good', 'great']

This is another way to write or creat rdd.

In [ ]:
temp = sc.parallelize([['a','b','c'],['d','e','f']])
temp.collect()

[['a', 'b', 'c'], ['d', 'e', 'f']]

**Map**
It is used as a maper of data from one state to another.

In [ ]:
rdd = text.map(lambda x : x.split(' '))

In [ ]:
rdd.collect()

[['Hi', 'How', 'are', 'you'],
 ['Hope', 'you', 'are', 'doing', 'good'],
 ['great']]

In [ ]:
def ex(x):
  l = x.split()
  f = []
  for i in l:
    f.append(i+'hello')
  return f
rdd1 = text.map(ex)

In [ ]:
rdd1.collect()

[['Hihello', 'Howhello', 'arehello', 'youhello'],
 ['Hopehello', 'youhello', 'arehello', 'doinghello', 'goodhello'],
 ['greathello']]

find the length of each word and return only length.

In [ ]:
def findlength(x):
  l = x.split(' ')
  f = ''
  for i in range(len(l)):
    if i!= len(l)-1:
      f = f + str(len(l[i])) + ' '
    else:
      f = f + str(len(l[i]))
  return f
rdd = text.map(findlength)

In [ ]:
rdd.collect()

['2 3 3 3', '4 3 3 5 4', '5']

**flatMap:**
 It is used as a mapper of data and explodes data before final output.

In [ ]:
rdd1 = rdd.flatMap(lambda x : x.split(' '))
rdd1.collect()

['2', '3', '3', '3', '4', '3', '3', '5', '4', '5']

**filter:** It is used to remove elements from rdd.

In [ ]:
rdd2 = rdd1.filter(lambda x : x!='2')
rdd2.collect()

['3', '3', '3', '4', '3', '3', '5', '4', '5']

In [ ]:
def ex(x):
  if x != '2':
    return True
  else:
    return False
rdd2 = rdd1.filter(ex)
rdd2.collect()

['3', '3', '3', '4', '3', '3', '5', '4', '5']

**Distinct:** It is used to get unique/distinct elements

In [ ]:
def ex(x):
  if x != '2':
    return True
  else:
    return False
rdd2 = rdd1.filter(ex)
rdd3 = rdd2.distinct()
rdd3.collect()

['4', '3', '5']

**Count:** It return no.of elemets in rdd.

In [ ]:
rdd2.count()

9

**CountByValue:** It provides how many times each value hass occur in rdd.

In [ ]:
rdd2.countByValue()

defaultdict(int, {'3': 5, '4': 2, '5': 2})

**GroupByValue:** It is used to create groups based on Keys in RDD.

In [ ]:
def findlen(x):
  return (x,len(x))
rdd4 = rdd1.map(findlen)
rdd4.groupByKey().mapValues(list).collect()

[('4', [1, 1]), ('2', [1]), ('3', [1, 1, 1, 1, 1]), ('5', [1, 1])]

**ReduceByKey:** It is used to combine data based on keys in rdd.

In [ ]:
rdd4.reduceByKey(lambda x,y:x+y).collect()

[('4', 2), ('2', 1), ('3', 5), ('5', 2)]

**Spark SQL**

If we read data from SprakContext then spark provides data as RDD.

If we read data from SpaekSession then spark provides data as dataframes.

In [2]:
from pyspark import SparkContext

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("py-Spark")\
        .getOrCreate()

In [6]:
!wget https://raw.githubusercontent.com/haneeth25/Libraries_prac/main/Student_performance_data%20_.csv

--2024-07-26 11:36:58--  https://raw.githubusercontent.com/haneeth25/Libraries_prac/main/Student_performance_data%20_.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166901 (163K) [text/plain]
Saving to: ‘Student_performance_data _.csv.1’

Student_performance 100%[===================>] 162.99K  --.-KB/s    in 0.03s   

2024-07-26 11:36:58 (4.79 MB/s) - ‘Student_performance_data _.csv.1’ saved [166901/166901]



Here we are creating dataframe df by using csv file "student_performance_data".

In [20]:
df = spark.read.option("header",True).csv('/content/Student_performance_data_.csv')

In [21]:
df.show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 17|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 18|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 15|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0| 

We will print Schema of the data frame by default it will take all datatupes as strings.

In [22]:
df.printSchema()

root
 |-- StudentID: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Ethnicity: string (nullable = true)
 |-- ParentalEducation: string (nullable = true)
 |-- StudyTimeWeekly: string (nullable = true)
 |-- Absences: string (nullable = true)
 |-- Tutoring: string (nullable = true)
 |-- ParentalSupport: string (nullable = true)
 |-- Extracurricular: string (nullable = true)
 |-- Sports: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- Volunteering: string (nullable = true)
 |-- GPA: string (nullable = true)
 |-- GradeClass: string (nullable = true)



If we want it to automatically take the datatype as the datatype in csv then we use this way.

In [24]:
df = spark.read.option('inferSchema',True).option('header',True).csv('/content/Student_performance_data_.csv')
# df.show()
df.printSchema()

root
 |-- StudentID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: integer (nullable = true)
 |-- Ethnicity: integer (nullable = true)
 |-- ParentalEducation: integer (nullable = true)
 |-- StudyTimeWeekly: double (nullable = true)
 |-- Absences: integer (nullable = true)
 |-- Tutoring: integer (nullable = true)
 |-- ParentalSupport: integer (nullable = true)
 |-- Extracurricular: integer (nullable = true)
 |-- Sports: integer (nullable = true)
 |-- Music: integer (nullable = true)
 |-- Volunteering: integer (nullable = true)
 |-- GPA: double (nullable = true)
 |-- GradeClass: double (nullable = true)



If we want to assign datatypes manually then we need to use this.

for this first we need to import StuctType,StructField,StringType,IntegerType.

In [15]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [23]:
s = StructType(
    [
        StructField('age',IntegerType(),True),
        StructField('Gender',IntegerType(),True),
        StructField('Ethnicity',IntegerType(),True),
    ]
)

df = spark.read.options(header='True').schema(s).csv('/content/Student_performance_data_.csv')
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- Gender: integer (nullable = true)
 |-- Ethnicity: integer (nullable = true)



**Selecting DataFrame Columns:**

In [27]:
df.select('gender').show()

+------+
|gender|
+------+
|     1|
|     0|
|     0|
|     1|
|     1|
|     0|
|     0|
|     1|
|     0|
|     1|
|     0|
|     0|
|     0|
|     0|
|     1|
|     0|
|     0|
|     1|
|     0|
|     0|
+------+
only showing top 20 rows



In [26]:
df.select('gender','age').show()

+------+---+
|gender|age|
+------+---+
|     1| 17|
|     0| 18|
|     0| 15|
|     1| 17|
|     1| 17|
|     0| 18|
|     0| 15|
|     1| 15|
|     0| 17|
|     1| 16|
|     0| 17|
|     0| 17|
|     0| 17|
|     0| 17|
|     1| 18|
|     0| 15|
|     0| 18|
|     1| 18|
|     0| 18|
|     0| 17|
+------+---+
only showing top 20 rows



when we give like this the column names become case sensitive.

In [31]:
df.select(df.Gender,df.Age).show()

+------+---+
|Gender|Age|
+------+---+
|     1| 17|
|     0| 18|
|     0| 15|
|     1| 17|
|     1| 17|
|     0| 18|
|     0| 15|
|     1| 15|
|     0| 17|
|     1| 16|
|     0| 17|
|     0| 17|
|     0| 17|
|     0| 17|
|     1| 18|
|     0| 15|
|     0| 18|
|     1| 18|
|     0| 18|
|     0| 17|
+------+---+
only showing top 20 rows



To get all columns.

In [32]:
df.select('*').show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 17|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 18|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 15|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0| 

If we want to know what all columns are there then we use this.

In [33]:
df.columns

['StudentID',
 'Age',
 'Gender',
 'Ethnicity',
 'ParentalEducation',
 'StudyTimeWeekly',
 'Absences',
 'Tutoring',
 'ParentalSupport',
 'Extracurricular',
 'Sports',
 'Music',
 'Volunteering',
 'GPA',
 'GradeClass']

I we want column name of 2nd columns or any particular number then we use this.

In [34]:
df.columns[2]

'Gender'

If we want 2nd column to show and we don't know the column name tehn we use this.

In [35]:
df.select(df.columns[2]).show()

+------+
|Gender|
+------+
|     1|
|     0|
|     0|
|     1|
|     1|
|     0|
|     0|
|     1|
|     0|
|     1|
|     0|
|     0|
|     0|
|     0|
|     1|
|     0|
|     0|
|     1|
|     0|
|     0|
+------+
only showing top 20 rows



If i want to show columns from 2nd to 4th then we use this. (column slicing staring point is inclusive but ending point is exclusive.)

In [36]:
df.select(df.columns[2:5]).show()

+------+---------+-----------------+
|Gender|Ethnicity|ParentalEducation|
+------+---------+-----------------+
|     1|        0|                2|
|     0|        0|                1|
|     0|        2|                3|
|     1|        0|                3|
|     1|        0|                2|
|     0|        0|                1|
|     0|        1|                1|
|     1|        1|                4|
|     0|        0|                0|
|     1|        0|                1|
|     0|        0|                1|
|     0|        0|                1|
|     0|        1|                1|
|     0|        1|                2|
|     1|        0|                1|
|     0|        0|                2|
|     0|        3|                1|
|     1|        0|                0|
|     0|        1|                3|
|     0|        0|                1|
+------+---------+-----------------+
only showing top 20 rows



To create new calumn by using existing column then we use **with column**

here iam creating a new column new_age by using existing column age.

For this first we need to import col.

In [38]:
from pyspark.sql.functions import col
df1 = df.withColumn('New_age',col('Age')+5)
df1.show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+-------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|New_age|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+-------+
|     1001| 17|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|     22|
|     1002| 18|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|     23|
|     1003| 15|     0|        2|                3|  4.21056976881226|      26|  

We can change values of a column by using it self instead of creating new column.

In [39]:
df = df.withColumn('Age',col('Age')+5)
df.show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 22|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 23|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 20|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0| 

We can change the values of a column without using aexisting column by using **lit.**

Here it will change all the values of age column to 23.

In [40]:
from pyspark.sql.functions import lit
df2 = df.withColumn('age',lit(23))
df2.show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 23|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 23|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 23|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0| 

If we want to change the age values to 23 whoes value = 17 then we use this.

In [43]:
from pyspark.sql.functions import when

df2 = df.withColumn('Age', when(df.Age == 22, lit(27)).otherwise(df.Age))
df2.show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 27|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 23|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 20|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0| 

Renaming the column.

In [44]:
df3 = df.withColumnRenamed('gender','sex')
df3.show()

+---------+---+---+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|sex|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+---+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 22|  1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 23|  0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 20|  0|        2|                3|  4.21056976881226|      26|       0|              2|              0|     0|    0|      

If we don't want to manipulate df instead we can use **alias**

It will not manipulate original df.

In [45]:
df.select(col('Gender').alias('Sex')).show()

+---+
|Sex|
+---+
|  1|
|  0|
|  0|
|  1|
|  1|
|  0|
|  0|
|  1|
|  0|
|  1|
|  0|
|  0|
|  0|
|  0|
|  1|
|  0|
|  0|
|  1|
|  0|
|  0|
+---+
only showing top 20 rows



This is the original df and it did not manipulate or change gender into sex.

In [46]:
df.show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 22|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1002| 23|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0| 3.042914833436377|       1.0|
|     1003| 20|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0| 

**Filter:**
It is used to get or select the df with certain condition.

In [48]:
df.filter(df.Age == 23).show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|                GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|     1002| 23|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0|     0|    0|           0|  3.042914833436377|       1.0|
|     1006| 23|     0|        0|                1| 8.191218545250186|       0|       0|              1|              1|     0|    0|           0| 3.0841836144863937|       1.0|
|     1015| 23|     1|        0|                1|11.197810636915708|       9|       1|              2|            

For multiple rows.

we can use:

-> & and operator

-> | or operator

-> == equal to operator. and so on..

In [49]:
df.filter((df.Age == 23) & (df.Gender == 0)).show()

+---------+---+------+---------+-----------------+-------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|    StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|                GPA|GradeClass|
+---------+---+------+---------+-----------------+-------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|     1002| 23|     0|        0|                1|  15.40875605584674|       0|       0|              1|              0|     0|    0|           0|  3.042914833436377|       1.0|
|     1006| 23|     0|        0|                1|  8.191218545250186|       0|       0|              1|              1|     0|    0|           0| 3.0841836144863937|       1.0|
|     1017| 23|     0|        3|                1| 10.098656081788002|      14|       0|              2|      

In [50]:
df.printSchema()

root
 |-- StudentID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: integer (nullable = true)
 |-- Ethnicity: integer (nullable = true)
 |-- ParentalEducation: integer (nullable = true)
 |-- StudyTimeWeekly: double (nullable = true)
 |-- Absences: integer (nullable = true)
 |-- Tutoring: integer (nullable = true)
 |-- ParentalSupport: integer (nullable = true)
 |-- Extracurricular: integer (nullable = true)
 |-- Sports: integer (nullable = true)
 |-- Music: integer (nullable = true)
 |-- Volunteering: integer (nullable = true)
 |-- GPA: double (nullable = true)
 |-- GradeClass: double (nullable = true)



We can also use like operator to check the pattern in strings.

In [ ]:
# it will show those values which has se in their value in name column asame in sql.
df.filter(df.name.like("%se%")).show()

In [ ]:
# it will show those values which has se in their value in name column.
df.filter(df.name,contains("se")).show()

-> Count = count the no.of rows.

-> Distinct = to get all different values.

-> drop duplicate = it will drop duplicate values.

In [51]:
df.filter(df.Age == '23').count()

582

In [60]:
# df.select(df.Age).distinct().count().show()
distinct_count = df.select(df.Age).distinct().count()
print(distinct_count)

4


Here we cannot use df.column_name format here in this drop duplicate.

In [61]:
df4 = df.dropDuplicates(["age"]).show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|               GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+------------------+----------+
|     1001| 22|     1|        0|                2|19.833722807854713|       7|       1|              2|              0|     0|    1|           0| 2.929195591667681|       2.0|
|     1003| 20|     0|        2|                3|  4.21056976881226|      26|       0|              2|              0|     0|    0|           0|0.1126022544661815|       4.0|
|     1002| 23|     0|        0|                1| 15.40875605584674|       0|       0|              1|              0| 

**sorting:**

In [62]:
df.sort('age').show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+--------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|                 GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+--------------------+----------+
|     1058| 20|     0|        0|                4| 10.87755842322235|      29|       0|              3|              1|     0|    0|           0|  0.2865712882866222|       4.0|
|     1138| 20|     0|        3|                2|7.2516642518935415|      18|       0|              0|              1|     0|    1|           1|  1.1829875004980481|       4.0|
|     1060| 20|     1|        3|                0| 9.468698671880976|      23|       0|              3|       

We can use multiple columns to sort.

In [63]:
df.sort(['age','Ethnicity']).show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|                GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|     1179| 20|     1|        0|                1| 3.368811022638305|       2|       0|              2|              1|     0|    0|           0|  2.553439811443938|       2.0|
|     1322| 20|     1|        0|                1| 17.00635416315198|      11|       0|              4|              1|     0|    0|           0|  2.627693905554347|       2.0|
|     1181| 20|     0|        0|                2|19.698582168829834|       5|       1|              3|            

**Order By:**

In [64]:
df.orderBy(df.Age).show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+--------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|                 GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+--------------------+----------+
|     1058| 20|     0|        0|                4| 10.87755842322235|      29|       0|              3|              1|     0|    0|           0|  0.2865712882866222|       4.0|
|     1138| 20|     0|        3|                2|7.2516642518935415|      18|       0|              0|              1|     0|    1|           1|  1.1829875004980481|       4.0|
|     1060| 20|     1|        3|                0| 9.468698671880976|      23|       0|              3|       

Ordering in Ascending order or decending order.

First we need to omport asc and desc.

In [69]:
from pyspark.sql.functions import asc
from pyspark.sql.functions import desc

In [72]:
df.orderBy(asc(df.Age)).show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+--------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|                 GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+--------------------+----------+
|     1058| 20|     0|        0|                4| 10.87755842322235|      29|       0|              3|              1|     0|    0|           0|  0.2865712882866222|       4.0|
|     1138| 20|     0|        3|                2|7.2516642518935415|      18|       0|              0|              1|     0|    1|           1|  1.1829875004980481|       4.0|
|     1060| 20|     1|        3|                0| 9.468698671880976|      23|       0|              3|       

In [73]:
df.orderBy(desc(df.Age)).show()

+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|StudentID|Age|Gender|Ethnicity|ParentalEducation|   StudyTimeWeekly|Absences|Tutoring|ParentalSupport|Extracurricular|Sports|Music|Volunteering|                GPA|GradeClass|
+---------+---+------+---------+-----------------+------------------+--------+--------+---------------+---------------+------+-----+------------+-------------------+----------+
|     1054| 23|     0|        0|                2|  8.46716054701572|      13|       0|              1|              0|     1|    0|           0| 1.6643106896610833|       4.0|
|     1121| 23|     1|        0|                3| 3.170404680548189|      12|       1|              1|              1|     1|    0|           0| 2.1749027962098797|       3.0|
|     1056| 23|     0|        0|                1| 8.847874814963872|      25|       0|              2|            